In [ ]:
!pip install autolgbm
!pip install pycaret

# COPIED FROM RISHIRAJ ACHARYA +2,-2 · 17H AGO · 1 VIEW · PRIVATE
# I appreciate it!
# your code is very useful and easy to understand!!

# In this notebook, I use pycaret library.
# paste below reference site url.
https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb

## version 6 
### refactering codes and make functions. <br>
### implement ensemble learning using pycaret ensemble functions.

In [ ]:
import pandas as pd
from lightgbm import LGBMRegressor
from autolgbm import AutoLGBM
from pycaret.regression import *
import random as r

In [ ]:
%rm -rf output

## read data

In [ ]:
# required parameters:
train_filename = "../input/mmlm22-men/train.csv"

# optional parameters
test_filename = "../input/mmlm22-men/test.csv"

train_df = pd.read_csv(train_filename)
# exp_name = setup(data = train_df,  target = 'Pred')

In [ ]:
# required parameters:
train_filename = "../input/mmlm22-men/train.csv"
output = "output"

# optional parameters
test_filename = "../input/mmlm22-men/test.csv"
task = None
idx = "ID"
targets = ["Pred"]
features = None
categorical_features = None
use_gpu = False
num_folds = 5
seed = 77
num_trials = 10000
time_limit = 7200
fast = False

seed = r.randint(1,100)

In [ ]:
# Now its time to train the model!
algbm = AutoLGBM(
    train_filename=train_filename,
    output=output,
    test_filename=test_filename,
    task=task,
    idx=idx,
    targets=targets,
    features=features,
    categorical_features=categorical_features,
    use_gpu=use_gpu,
    num_folds=num_folds,
    seed=seed,
    num_trials=num_trials,
    time_limit=time_limit,
    fast=fast,
)

# algbm.train()

In [ ]:
# submission = pd.read_csv("./output/test_predictions.csv")
# submission.drop('0.0', inplace=True, axis=1)
# submission.rename(columns = {'1.0':'Pred'}, inplace = True)


## separete learning dataset and test data
## you can set learning data ratio.
## for example, in this below, you can see frac parameter 0.8 which is used learning data 80% all of train data.

In [ ]:
data = train_df.copy()
# data_unseen = data.drop(data.index)

data.reset_index(drop=True, inplace=True)
# data_unseen.reset_index(drop=True, inplace=True)

## Read train dara and set up target

In [ ]:
exp_reg101 = setup(data = data, target = 'Pred', session_id=123,silent = True,feature_selection = True) 

## AutoML
## run compare_models, a lot of machine learning model are start learning.
## you can see 6 metrics after modeling.
## good model which is low error metrics is painted yellow.

In [ ]:
best = compare_models()
# top5 = compare_models(n_select = 5)
# blender = stack_models(top5)

In [ ]:
params={'reg_alpha': 0.0707367246786973, 'reg_lambda': 0.06928113227468818, 
        'colsample_bytree': 0.647565718092968, 'subsample': 0.44011277781856156, 
        'learning_rate': 0.0052795751704346455, 'max_depth': 100, 'num_leaves': 356, 
        'min_child_samples': 72}

Y = data['Pred']
X = data.drop(['Pred','ID', 'IDTeams_c_score'],axis=1)
model = LGBMRegressor(**params)
model.fit(X,Y)


## create_model
## select model.
## it does cross_validation and you can confirm each metricses.

In [ ]:
# train individual models to blend
gbr = create_model('gbr', verbose = False)
dt = create_model('dt', verbose = False)
en = create_model('en', verbose = False)
rf = create_model('rf', verbose = False)
et = create_model('et', verbose = False)

# blend individual models
## 

In [ ]:
# blender = blend_models(best)
blender = blend_models(estimator_list = [gbr,dt,en,rf,et])

## model tuning

In [ ]:
tuned_model = tune_model(blender)

## prediction
## set prediction model
## after setting, automatically display model name, metricses and predict data.

In [ ]:
predict_model(tuned_model)

## finalize model

In [ ]:
 final_model = finalize_model(tuned_model)

## confirm hyper parameter

In [ ]:
final_model

In [ ]:
# top5 = compare_models(n_select = 5)
# stacker = stack_models(top5)

## prediction

In [ ]:
test_df = pd.read_csv(test_filename)
unseen_predictions = predict_model(final_model, data=test_df)
test_df2 = test_df.drop(['ID', 'IDTeams_c_score'],axis=1)
preds3 = model.predict(test_df2)

In [ ]:
unseen_predictions

## prepare submmsion data

In [ ]:
sub = unseen_predictions[['ID', 'Label']].copy()
sub = sub.rename(columns={'Label':'Pred'})
sub.head()

In [ ]:
# sub2 = sub.copy()
# sub2['Pred'] = preds3
# sub2.head()

In [ ]:
# sub['Pred'] = (sub['Pred']+sub2['Pred'])/2.
# sub['Pred'].head()

In [ ]:
sub.to_csv('submission.csv',index=False)